### Loading in the data



In [16]:
import matplotlib.pyplot as plt
#import statsmodels.tsa.seasonal as smt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import datetime as dt


In [17]:
#read data
#takes 1 minute for 1000 files

import os 
from os.path import expanduser
home = expanduser("~")
stock_data_dir=os.path.join(home,'Desktop\Deep Learning Project\Data\price-volume-data-for-all-us-stocks-etfs\Data\Stocks')
filenames = [x for x in os.listdir(stock_data_dir) if x.endswith('.txt') and os.path.getsize(os.path.join(stock_data_dir,x)) > 0]
#filenames = ['a.us.txt', aa.us.txt, 'bgr.us.txt']
filenames=filenames[:100]


#data = []
result=None
for filename in filenames:
    df = pd.read_csv(os.path.join(stock_data_dir,filename), sep=',')

    label, _, _ = filename.split(sep='.')
    df.insert(loc=0, column='Label', value=filename)
    df['Date'] = pd.to_datetime(df['Date'])
    #Response Variable
    df['ReturnOpenNext1']=(df.Open.shift(-1)-df.Open)/df.Open
    #Features
    df['ReturnOpenPrevious1']=(df.Open-df.Open.shift(1))/df.Open.shift(1)
    df['ReturnOpenPrevious2']=(df.Open-df.Open.shift(2))/df.Open.shift(2)
    df['ReturnOpenPrevious5']=(df.Open-df.Open.shift(5))/df.Open.shift(5)
    
    #data.append(df)
    if type(result)==type(None):
        result=df
    else:
        result=result.append(df,ignore_index=True)
    
result.head()

,Label,Date,Open,High,Low,Close,Volume,OpenInt,ReturnOpenNext1,ReturnOpenPrevious1,ReturnOpenPrevious2,ReturnOpenPrevious5
0,a.us.txt,1999-11-18,30.713,33.754,27.002,29.702,66277506,0,-0.056230,NaN,NaN,NaN
1,a.us.txt,1999-11-19,28.986,29.027,26.872,27.257,16142920,0,-0.037949,-0.056230,NaN,NaN
2,a.us.txt,1999-11-22,27.886,29.702,27.044,29.702,6970266,0,0.028760,-0.037949,-0.092046,NaN
3,a.us.txt,1999-11-23,28.688,29.446,27.002,27.002,6332082,0,-0.055947,0.028760,-0.010281,NaN
4,a.us.txt,1999-11-24,27.083,28.309,27.002,27.717,5132147,0,0.018868,-0.055947,-0.028796,NaN


In [36]:
result1=result.copy()
encoder_Label = dict()
encoder_Label = {l: id for id, l in enumerate(filenames)}

result1.Label=[encoder_Label.get(i) for i in result1.Label]
result1.head()

,Label,Date,Open,High,Low,Close,Volume,OpenInt,ReturnOpenNext1,ReturnOpenPrevious1,ReturnOpenPrevious2,ReturnOpenPrevious5
0,0,1999-11-18,30.713,33.754,27.002,29.702,66277506,0,-0.056230,NaN,NaN,NaN
1,0,1999-11-19,28.986,29.027,26.872,27.257,16142920,0,-0.037949,-0.056230,NaN,NaN
2,0,1999-11-22,27.886,29.702,27.044,29.702,6970266,0,0.028760,-0.037949,-0.092046,NaN
3,0,1999-11-23,28.688,29.446,27.002,27.002,6332082,0,-0.055947,0.028760,-0.010281,NaN
4,0,1999-11-24,27.083,28.309,27.002,27.717,5132147,0,0.018868,-0.055947,-0.028796,NaN


In [37]:
result2=result1.copy()
encoder_Date = dict()
encoder_Date = {l: id for id, l in enumerate(np.sort(result2.Date.unique()))}
result2.Date=[encoder_Date.get(i) for i in result2.Date.values]
result2.head()

,Label,Date,Open,High,Low,Close,Volume,OpenInt,ReturnOpenNext1,ReturnOpenPrevious1,ReturnOpenPrevious2,ReturnOpenPrevious5
0,0,7552,30.713,33.754,27.002,29.702,66277506,0,-0.056230,NaN,NaN,NaN
1,0,7553,28.986,29.027,26.872,27.257,16142920,0,-0.037949,-0.056230,NaN,NaN
2,0,7554,27.886,29.702,27.044,29.702,6970266,0,0.028760,-0.037949,-0.092046,NaN
3,0,7555,28.688,29.446,27.002,27.002,6332082,0,-0.055947,0.028760,-0.010281,NaN
4,0,7556,27.083,28.309,27.002,27.717,5132147,0,0.018868,-0.055947,-0.028796,NaN


In [38]:
#result3=np.empty((len(encoder_Label),len(encoder_Date),6))
#result3[:,:,:]=None
#result3.shape
#for index, row in result2.iterrows():
#   result3[int(row['Label']),int(row['Date']),:]=row[['Open','High','Low','Close','Volume','OpenInt']]
#result3

result3=np.empty((len(encoder_Label),len(encoder_Date),result2.shape[1]-2))
result3[:,:,:]=None

for row in result2.itertuples():
    result3[int(row.Label),int(row.Date),:]=row[3:]
result3[0,7552,:]


array([ 3.07130000e+01,  3.37540000e+01,  2.70020000e+01,  2.97020000e+01,
        6.62775060e+07,  0.00000000e+00, -5.62302608e-02,             nan,
                   nan,             nan])

### Implementaion Baseline Model
### Starting from result2 (Labels and Dates encoded

In [39]:
result2.head()

,Label,Date,Open,High,Low,Close,Volume,OpenInt,ReturnOpenNext1,ReturnOpenPrevious1,ReturnOpenPrevious2,ReturnOpenPrevious5
0,0,7552,30.713,33.754,27.002,29.702,66277506,0,-0.056230,NaN,NaN,NaN
1,0,7553,28.986,29.027,26.872,27.257,16142920,0,-0.037949,-0.056230,NaN,NaN
2,0,7554,27.886,29.702,27.044,29.702,6970266,0,0.028760,-0.037949,-0.092046,NaN
3,0,7555,28.688,29.446,27.002,27.002,6332082,0,-0.055947,0.028760,-0.010281,NaN
4,0,7556,27.083,28.309,27.002,27.717,5132147,0,0.018868,-0.055947,-0.028796,NaN


In [40]:
#Filtering out none rows
market_df=result2.loc[~result2.isnull().any(axis=1)]
market_df.head()

,Label,Date,Open,High,Low,Close,Volume,OpenInt,ReturnOpenNext1,ReturnOpenPrevious1,ReturnOpenPrevious2,ReturnOpenPrevious5
5,0,7557,27.594,28.012,27.509,27.807,1832635,0,0.002972,0.018868,-0.038134,-0.101553
6,0,7558,27.676,28.650,27.380,28.432,4317826,0,0.024353,0.002972,0.021896,-0.045194
7,0,7559,28.350,28.986,27.634,28.480,4567146,0,0.004586,0.024353,0.027397,0.016639
8,0,7560,28.480,29.324,28.273,28.986,3133746,0,0.036938,0.004586,0.029050,-0.007250
9,0,7561,29.532,30.375,29.155,29.786,3252997,0,0.027225,0.036938,0.041693,0.090426


In [41]:
#Sorting in Date order
market_df=market_df.sort_values(['Date','Label'])
market_df.reset_index(inplace=True,drop=True)
market_df.head()

,Label,Date,Open,High,Low,Close,Volume,OpenInt,ReturnOpenNext1,ReturnOpenPrevious1,ReturnOpenPrevious2,ReturnOpenPrevious5
0,1,5,2.2139,2.2139,2.1921,2.1921,72418,0,-0.009847,-0.009086,-0.009086,-0.036262
1,1,6,2.1921,2.1921,2.1717,2.1717,94257,0,-0.009306,-0.009847,-0.018843,-0.045751
2,1,7,2.1717,2.1921,2.1717,2.1921,67818,0,0.009394,-0.009306,-0.019061,-0.045784
3,1,8,2.1921,2.1921,2.1717,2.1717,66669,0,-0.009306,0.009394,0.000000,-0.018843
4,1,9,2.1717,2.1921,2.1717,2.1921,91958,0,0.009394,-0.009306,0.000000,-0.027974


In [42]:
from sklearn.model_selection import train_test_split

market_train_indices, market_val_indices = train_test_split(market_df.index,test_size=0.25, random_state=23)

# Handling numerical variables

In [45]:
from sklearn.preprocessing import StandardScaler
response_col=market_df.columns.get_loc("ReturnOpenNext1")
scaler = StandardScaler()
num_feature_cols=list(market_df.columns[response_col+1:])
market_df[num_feature_cols] = scaler.fit_transform(market_df[num_feature_cols])
market_df[:10]

,Label,Date,Open,High,Low,Close,Volume,OpenInt,ReturnOpenNext1,ReturnOpenPrevious1,ReturnOpenPrevious2,ReturnOpenPrevious5
0,1,5,2.2139,2.2139,2.1921,2.1921,72418,0,-0.009847,-0.193202,-0.166552,-0.422057
1,1,6,2.1921,2.1921,2.1717,2.1717,94257,0,-0.009306,-0.207856,-0.316939,-0.521465
2,1,7,2.1717,2.1921,2.1717,2.1921,67818,0,0.009394,-0.197441,-0.320298,-0.521808
3,1,8,2.1921,2.1921,2.1717,2.1717,66669,0,-0.009306,0.162719,-0.026514,-0.239599
4,1,9,2.1717,2.1921,2.1717,2.1921,91958,0,0.009394,-0.197441,-0.026514,-0.335246
5,1,10,2.1921,2.1921,2.1717,2.1921,105750,0,0.000000,0.162719,-0.026514,-0.145359
6,1,11,2.1921,2.1921,2.1717,2.1717,109200,0,-0.009306,-0.018203,0.118265,-0.042211
7,1,12,2.1717,2.1921,2.1717,2.1921,22991,0,0.009394,-0.197441,-0.169945,-0.042211
8,1,13,2.1921,2.2342,2.1921,2.2342,37931,0,0.019205,0.162719,-0.026514,-0.042211
9,1,14,2.2342,2.2972,2.2342,2.2544,212652,0,0.009041,0.351696,0.417048,0.259257


# Define NN Architecture

In [46]:
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Concatenate, Flatten, BatchNormalization
from keras.losses import binary_crossentropy, mse

embedding_dimension=10

label_input=Input(shape=[1], name="label_input")
label_embedding=Embedding(len(encoder_Label), embedding_dimension)(label_input)
label_logits = Flatten()(label_embedding)
label_logits = Dense(32,activation='relu')(label_logits)

numerical_inputs = Input(shape=(len(num_feature_cols),), name='num_input')
numerical_logits = numerical_inputs
numerical_logits = BatchNormalization()(numerical_logits)   #I do not think this makes sense, since we scaler.fit_transform
numerical_logits = Dense(128,activation='relu')(numerical_logits)
numerical_logits = Dense(64,activation='relu')(numerical_logits)

logits = Concatenate()([numerical_logits,label_logits])
logits = Dense(64,activation='relu')(logits)
#out = Dense(1, activation='sigmoid')(logits)
out = Dense(1, )(logits)
model = Model(inputs = [label_input,numerical_inputs], outputs=out)

#model.compile(optimizer='adam',loss=binary_crossentropy)
model.compile(optimizer='adam',loss='mean_squared_error')

Using TensorFlow backend.


In [47]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
num_input (InputLayer)          (None, 3)            0                                            
__________________________________________________________________________________________________
label_input (InputLayer)        (None, 1)            0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 3)            12          num_input[0][0]                  
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 10)        1000        label_input[0][0]                
__________________________________________________________________________________________________
dense_2 (D

In [48]:
def get_input(market_df, indices):
    X_num = market_df.loc[indices, num_feature_cols].values
    X = {'num_input':X_num}
    X['label_input'] = market_df.loc[indices, 'Label'].values
    y = market_df.loc[indices,'ReturnOpenNext1'].values
    return X,y,

# r, u and d are used to calculate the scoring metric
X_train,y_train = get_input(market_df, market_train_indices)
X_valid,y_valid = get_input(market_df, market_val_indices)

In [87]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

check_point = ModelCheckpoint('model.hdf5',verbose=True, save_best_only=True)
early_stop = EarlyStopping(patience=5,verbose=True)
model.fit(X_train,y_train.astype(int),
          validation_data=(X_valid,y_valid.astype(int)),
          epochs=6,
          verbose=True,
          callbacks=[early_stop,check_point]) 

Train on 195214 samples, validate on 65072 samples
Epoch 1/6
195214/195214 [==============================] - 8s 42us/step - loss: 1.4424e-04 - val_loss: 0.0046

Epoch 00001: val_loss improved from inf to 0.00463, saving model to model.hdf5
Epoch 2/6
195214/195214 [==============================] - 8s 42us/step - loss: 1.4440e-04 - val_loss: 0.0046

Epoch 00002: val_loss improved from 0.00463 to 0.00463, saving model to model.hdf5
Epoch 3/6
195214/195214 [==============================] - 9s 44us/step - loss: 1.4415e-04 - val_loss: 0.0046

Epoch 00003: val_loss did not improve from 0.00463
Epoch 4/6
195214/195214 [==============================] - 8s 42us/step - loss: 1.4420e-04 - val_loss: 0.0046

Epoch 00004: val_loss did not improve from 0.00463
Epoch 5/6
195214/195214 [==============================] - 8s 42us/step - loss: 1.4396e-04 - val_loss: 0.0046

Epoch 00005: val_loss did not improve from 0.00463
Epoch 6/6
195214/195214 [==============================] - 8s 42us/step - loss:

In [51]:
os.getcwd()

'C:\\Users\\Thomas\\Desktop\\Deep Learning Project\\deeplearning\\dl'

# Evaluation of Validation Set



In [96]:
from sklearn.metrics import accuracy_score
predict_valid = model.predict(X_valid)
predict_train=model.predict(X_train)

In [153]:
print(accuracy_score(predict_valid[:,0]>=0,y_valid>=0))
accuracy_score(predict_train[:,0]>=0,y_train>=0)

0.47681030243422673


0.47684080035243376

## Bad results. Try different loss function (original loss)

In [158]:


embedding_dimension=10

label_input=Input(shape=[1], name="label_input")
label_embedding=Embedding(len(encoder_Label), embedding_dimension)(label_input)
label_logits = Flatten()(label_embedding)
label_logits = Dense(32,activation='relu')(label_logits)

numerical_inputs = Input(shape=(len(num_feature_cols),), name='num_input')
numerical_logits = numerical_inputs
numerical_logits = BatchNormalization()(numerical_logits)   #I do not think this makes sense, since we scaler.fit_transform 
numerical_logits = Dense(128,activation='relu')(numerical_logits)
numerical_logits = Dense(64,activation='relu')(numerical_logits)

logits = Concatenate()([numerical_logits,label_logits])
logits = Dense(64,activation='relu')(logits)
out = Dense(1, activation='sigmoid')(logits)
#out = Dense(1, )(logits)
model = Model(inputs = [label_input,numerical_inputs], outputs=out)

model.compile(optimizer='adam',loss=binary_crossentropy)
#model.compile(optimizer='adam',loss='mean_squared_error')

In [164]:
def get_input2(market_df, indices):
    X_num = market_df.loc[indices, num_feature_cols].values
    X = {'num_input':X_num}
    X['label_input'] = market_df.loc[indices, 'Label'].values
    y = (market_df.loc[indices,'ReturnOpenNext1']>= 0).values
    return X,y,

# r, u and d are used to calculate the scoring metric
X_train,y_train = get_input2(market_df, market_train_indices)
X_valid,y_valid = get_input2(market_df, market_val_indices)

In [169]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

check_point = ModelCheckpoint('model.hdf5',verbose=True, save_best_only=True)
early_stop = EarlyStopping(patience=5,verbose=True)
model.fit(X_train,y_train.astype(int),
          validation_data=(X_valid,y_valid.astype(int)),
          epochs=6,
          verbose=True,
          callbacks=[early_stop,check_point]) 

Train on 195214 samples, validate on 65072 samples
Epoch 1/6
195214/195214 [==============================] - 9s 49us/step - loss: 0.6903 - val_loss: 0.6891

Epoch 00001: val_loss improved from inf to 0.68906, saving model to model.hdf5
Epoch 2/6
195214/195214 [==============================] - 9s 48us/step - loss: 0.6892 - val_loss: 0.6889

Epoch 00002: val_loss improved from 0.68906 to 0.68894, saving model to model.hdf5
Epoch 3/6
195214/195214 [==============================] - 12s 61us/step - loss: 0.6889 - val_loss: 0.6888

Epoch 00003: val_loss improved from 0.68894 to 0.68876, saving model to model.hdf5
Epoch 4/6
195214/195214 [==============================] - 12s 60us/step - loss: 0.6887 - val_loss: 0.6889

Epoch 00004: val_loss did not improve from 0.68876
Epoch 5/6
195214/195214 [==============================] - 12s 60us/step - loss: 0.6886 - val_loss: 0.6887

Epoch 00005: val_loss improved from 0.68876 to 0.68873, saving model to model.hdf5
Epoch 6/6
195214/195214 [=======

In [174]:
predict_valid = model.predict(X_valid)[:,0]*2 -1
predict_train=model.predict(X_train)[:,0]*2 -1

In [202]:
print(accuracy_score(predict_train>0,y_train>0))
print(accuracy_score(predict_valid>0,y_valid>0))

0.5392338664235147
0.5374815588886157
